# Minimal Example

The aim of this notebook is to provide a minimal example how fides can be used to optimize user-defined functions. In this example, we will minimize the [Rosenbrock](https://en.wikipedia.org/wiki/Rosenbrock_function) function. First, we import the rosenbrock function an its derivatives from `scipy.optimize`.

In [1]:
from scipy.optimize import rosen, rosen_der, rosen_hess

Next, we define an objective function that returns a triple with function value, gradient and hessian.

In [2]:
def obj(x):
    return rosen(x), rosen_der(x), rosen_hess(x)

To optimize this function, we first create a `fides.Optimizer` instance based on the objective function defined above. The optimizer also requires upper and lower boundaries for optimization variables $x$. In this example, each optimization variable is only bounded in one direction, with $1.5 \leq x_0 \lt \infty$ and $-\infty \lt x_1 \leq -1.5$. These bounds must be passed as numpy arrays.

In [3]:
import fides
import numpy as np

opt = fides.Optimizer(obj, 
                      ub=np.asarray([np.inf, 1.5]), 
                      lb=np.asarray([-1.5, -np.inf]))

To perform optimization, we call the `minimize` method and pass the origin `(0, 0)` as starting point.

In [4]:
opt_f, opt_x, opt_grad, opt_hess = opt.minimize(np.asarray([0, 0]))

2022-01-11 16:13:17 fides(INFO)  iter|    fval   |   fdiff  | tr ratio |tr radius|  ||g||  | ||step||| step|acc
2022-01-11 16:13:17 fides(INFO)     0| +1.00E+00 |    NaN   |    NaN   | 1.0E+00 | 2.0E+00 |   NaN   | NaN |1
2022-01-11 16:13:17 fides(INFO)     1| +1.00E+00 | +9.9E+01 | -9.9E+01 | 1.0E+00 | 2.0E+00 | 1.0E+00 |  2d |0
2022-01-11 16:13:17 fides(INFO)     2| +9.53E-01 | -4.7E-02 | +1.1E-01 | 2.5E-01 | 2.0E+00 | 2.5E-01 |  2d |1
2022-01-11 16:13:17 fides(INFO)     3| +5.24E-01 | -4.3E-01 | +1.1E+00 | 6.2E-02 | 1.3E+01 | 7.7E-02 |  2d |1
2022-01-11 16:13:17 fides(INFO)     4| +3.92E-01 | -1.3E-01 | +9.4E-01 | 1.2E-01 | 1.4E+00 | 1.3E-01 |  2d |1
2022-01-11 16:13:17 fides(INFO)     5| +2.63E-01 | -1.3E-01 | +1.1E+00 | 2.5E-01 | 3.1E+00 | 1.7E-01 |  2d |1
2022-01-11 16:13:17 fides(INFO)     6| +1.74E-01 | -8.9E-02 | +1.4E+00 | 2.5E-01 | 4.2E+00 | 1.2E-01 |  2d |1
2022-01-11 16:13:17 fides(INFO)     7| +1.10E-01 | -6.4E-02 | +2.2E-01 | 2.5E-01 | 1.5E+00 | 2.0E-01 |  2d |1
2022-01-

During optimization, fides prints a series of diagnostic variables that can be accessed that may help the user to debug optimization. For example, here we can see that fides took 20 iterations for minimization (`iter` column), that the trust region radius was set to values between 1.0 and 0.81 (`tr radius` column) and that only two step proposals were rejected (`acc` column).

To verify that fides found the correct optimum, we can compare the returned values against reference values (we know that the rosenbrock function has it's minimum at $(1.0, 1.0)$ with function value $0.0$).

In [5]:
assert np.allclose(opt_x, [1.0, 1.0])
assert np.isclose(opt_f, 0.0)

To numerically verify that we found a local minimum, we can check whether the gradient is small and whether the Hessian has strictly positive eigenvalues.

In [6]:
assert np.allclose(opt_grad, [0.0, 0.0], atol=1e-7)
assert np.min(np.linalg.eig(opt_hess)[0]) > 0